# A.4b - RGB + Synthetic Depth (4-Channel) Training

**Experiment:** A.4b  
**Input:** RGB + Synthetic Depth (4-channel RGBD)  
**Objective:** Test fusion of RGB with synthetic depth (Depth-Anything-V2)  
**Classes:** 1 (fresh_fruit_bunch)

## Prerequisites
1. Run `generate_synthetic_depth.ipynb` first
2. Upload RGBD dataset as: `ffb-rgbd-synthetic`

## Workflow
1. Load RGB+Synthetic depth dataset (4-channel images)
2. Train YOLOv11n with 5 seeds (42, 123, 456, 789, 101)
3. Evaluate on test set
4. Calculate mean ± std deviation

## Training Config
- Model: YOLOv11n
- Epochs: 100
- Patience: 30
- Other parameters: default

In [ ]:
# =============================================================================
# Cell 1: Setup
# =============================================================================
import os
import torch
import numpy as np
import shutil
from pathlib import Path
from datetime import datetime

IS_KAGGLE = os.path.exists('/kaggle/input')
print(f"Running on: {'Kaggle' if IS_KAGGLE else 'Local'}")

# Paths
DATASET_PATH = Path('/kaggle/input/ffb-rgbd-synthetic')
BASE_PATH = Path('/kaggle/working')
RUNS_PATH = BASE_PATH / 'runs' / 'detect'
KAGGLE_OUTPUT = BASE_PATH / 'kaggleoutput'
KAGGLE_OUTPUT.mkdir(parents=True, exist_ok=True)

print(f"Dataset: {DATASET_PATH} (exists: {DATASET_PATH.exists()})")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

# Verify dataset
for split in ['train', 'val', 'test']:
    imgs = len(list((DATASET_PATH / 'images' / split).glob('*.png')))
    lbls = len(list((DATASET_PATH / 'labels' / split).glob('*.txt')))
    print(f"  {split}: {imgs} images, {lbls} labels")

In [ ]:
%%writefile /kaggle/working/dataset_rgbd_synthetic.yaml
# A.4b RGB+Synthetic Depth 4-Channel Dataset
path: /kaggle/input/ffb-rgbd-synthetic
train: images/train
val: images/val
test: images/test

nc: 1
names: ['fresh_fruit_bunch']

In [ ]:
# =============================================================================
# Cell 3: Verify YAML and Check Image Channels
# =============================================================================
import cv2

config_path = Path('/kaggle/working/dataset_rgbd_synthetic.yaml')
print(f"YAML: {config_path}")
print(config_path.read_text())

# Verify 4-channel images
sample_img = list((DATASET_PATH / 'images' / 'train').glob('*.png'))[0]
img = cv2.imread(str(sample_img), cv2.IMREAD_UNCHANGED)
print(f"\nSample image shape: {img.shape}")
assert img.shape[2] == 4, f"Expected 4 channels, got {img.shape[2]}"

In [ ]:
# =============================================================================
# Cell 4: Install Ultralytics
# =============================================================================
!pip install -q ultralytics

from ultralytics import YOLO
import pandas as pd
print("Ultralytics ready")

In [ ]:
# =============================================================================
# Cell 5: Training Config
# =============================================================================
SEEDS = [42, 123, 456, 789, 101]
EXP_PREFIX = 'exp_a4b_rgbd_synth'

print(f"Seeds: {SEEDS} ({len(SEEDS)} runs)")
print("Metrics: mean ± std deviation")
print("Note: 4-channel input (RGB + Synthetic Depth)")

In [ ]:
# =============================================================================
# Cell 6: Training Loop (4-channel)
# =============================================================================
for seed in SEEDS:
    print(f"\n{'='*60}")
    print(f"TRAINING A.4b RGBD SYNTHETIC - Seed {seed} ({SEEDS.index(seed)+1}/{len(SEEDS)})")
    print(f"{'='*60}\n")
    
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
    
    model = YOLO('yolo11n.pt')
    
    results = model.train(
        data=str(config_path),
        epochs=100,
        patience=30,
        seed=seed,
        name=f"{EXP_PREFIX}_seed{seed}",
        exist_ok=True,
        imgsz=640,
    )
    
    print(f"\nSeed {seed} complete!")
    print(f"mAP50: {results.results_dict.get('metrics/mAP50(B)', 0):.3f}")

In [ ]:
# =============================================================================
# Cell 7: Evaluation on Test Set
# =============================================================================
results_dict = {}

print("="*60)
print("EVALUATION ON TEST SET")
print("="*60)

for seed in SEEDS:
    model_path = RUNS_PATH / f"{EXP_PREFIX}_seed{seed}" / 'weights' / 'best.pt'
    
    if not model_path.exists():
        print(f"Not found: {model_path}")
        continue
    
    print(f"\nSeed {seed}:")
    model = YOLO(str(model_path))
    metrics = model.val(data=str(config_path), split='test')
    
    results_dict[seed] = {
        'mAP50': metrics.box.map50,
        'mAP50-95': metrics.box.map,
        'Precision': metrics.box.mp,
        'Recall': metrics.box.mr
    }
    
    print(f"  mAP50: {metrics.box.map50:.3f}")
    print(f"  mAP50-95: {metrics.box.map:.3f}")
    print(f"  Precision: {metrics.box.mp:.3f}")
    print(f"  Recall: {metrics.box.mr:.3f}")

In [ ]:
# =============================================================================
# Cell 8: Results Summary
# =============================================================================
df = pd.DataFrame(results_dict).T
df.index.name = 'Seed'

# Calculate mean and std
avg = df.mean()
std = df.std()

print("\n" + "="*60)
print("A.4b RGB+SYNTHETIC DEPTH - FINAL RESULTS")
print("="*60 + "\n")
print(df.to_string(float_format=lambda x: f"{x:.3f}"))

print("\n" + "-"*60)
print("SUMMARY (Mean ± Std)")
print("-"*60)
for col in df.columns:
    print(f"  {col}: {avg[col]:.3f} ± {std[col]:.3f}")

In [ ]:
# =============================================================================
# Cell 9: Save Results
# =============================================================================
output_file = KAGGLE_OUTPUT / 'a4b_rgbd_synthetic_results.txt'

with open(output_file, 'w') as f:
    f.write("="*60 + "\n")
    f.write("A.4b RGB+Synthetic Depth (4-Channel) Results\n")
    f.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write("Training: epochs=100, patience=30, other=default\n")
    f.write(f"Seeds: {SEEDS}\n")
    f.write("="*60 + "\n\n")
    f.write("Per-Seed Results:\n")
    f.write(df.to_string(float_format=lambda x: f"{x:.3f}"))
    f.write("\n\n" + "-"*60 + "\n")
    f.write("Summary (Mean ± Std):\n")
    for col in df.columns:
        f.write(f"  {col}: {avg[col]:.3f} ± {std[col]:.3f}\n")

print(f"Results saved: {output_file}")

In [ ]:
# =============================================================================
# Cell 10: Create Archives
# =============================================================================
if RUNS_PATH.exists():
    shutil.make_archive('/kaggle/working/a4b_runs', 'zip', RUNS_PATH)
    print(f"a4b_runs.zip: {os.path.getsize('/kaggle/working/a4b_runs.zip')/1024/1024:.1f} MB")

shutil.make_archive('/kaggle/working/a4b_output', 'zip', KAGGLE_OUTPUT)
print("a4b_output.zip created")

print("\nDownload from Output tab")